In [1]:
import json
import uuid
from AI_Functions import call_llm
from RAG_Functions import create_collection,create_vectors
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

llm_model="gpt-3.5-turbo"
Real_Estate_Listings = create_collection("Real_Estate")

In [2]:
Real_Estate_Listings.count()

0

In [3]:
# Define the function that will injects the list into the chroma DB
def generate_listing(Neighborhood, Price, Bedrooms, Bathrooms, House_Size, Description, Neighborhood_Description):
    print(Neighborhood, Price, Bedrooms, Bathrooms, House_Size, Description, Neighborhood_Description)
    Description_Vectors = create_vectors(Description,EMBEDDING_MODEL_NAME)
    Neighborhood_Description_Vectors = create_vectors(Neighborhood_Description,EMBEDDING_MODEL_NAME)
    property_id = str(uuid.uuid4())
    Real_Estate_Listings.upsert(
    documents=[Description,Neighborhood_Description],
    metadatas=[{"property_ID":property_id,"Doc_type" : "Description" , "Neighborhood": Neighborhood, "Price": Price , "Bedrooms": Bedrooms, "Bathrooms": Bathrooms, "House_Size": House_Size },{"property_ID":property_id,"Doc_type" : "Neighborhood_Description", "Neighborhood": Neighborhood, "Price": Price , "Bedrooms": Bedrooms, "Bathrooms": Bathrooms, "House_Size": House_Size}],
    ids=[str(uuid.uuid4()), str(uuid.uuid4())],
    embeddings=[Description_Vectors,Neighborhood_Description_Vectors]
    )
    return f"{property_id} has been added to the Real Estate Listings"


available_functions = {
        "generate_listing": generate_listing,
    }

In [4]:
# Define the tools that will be available to the user
tools=[
    {
        "type": "function",
        "function": {
            "name": "generate_listing",
            "description": "Create a real estate Listing",
            "parameters": {
                "type": "object",
                "properties": {
                    "Neighborhood": {
                        "type": "string",
                        "description": "Neighborhood in Victoria Australia. E.g. Williams Landing"
                    },
                    "Price": {
                        "type": "string",
                        "description": "Random Price of the House in the range between $600,000 and $2,000,000. E.g. $800,000"
                    },
                    "Bedrooms": {
                        "type": "number",
                        "description": "Range from 2 to 5. E.g. 3"
                    },
                    "Bathrooms": {
                        "type": "number",
                        "description": "Range from 1 to 3. E.g. 2"
                    },
                    "House_Size": {
                        "type": "string",
                        "description": "Range from 1000sqft to 3000sqft. E.g. 2000sqft"
                    },
                    "Description": {
                        "type": "string",
                        "description": "Factual House Description explaining the features. e.g. Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem."
                    },
                    "Neighborhood_Description": {
                        "type": "string",
                        "description": "Description of Neighborhood explaning the access to commute, grocery, Restaurants, schools etc. e.g. Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze"
                    }
                },
                "required": ["Neighborhood","Price","Bedrooms","Bathrooms","House_Size","Description","Neighborhood_Description"]
            }
        }
    }
    ]

In [5]:
while Real_Estate_Listings.count() < 30: # Creating 15 listings, Since I am loading Description and Neighborhood Description as separate documents, I am creating 30 documents
    messages = [{"role": "user", "content": "create 10 diverse list of real estate listing which is not same as example"}] # Creating one by one is leading to repeated listings, so creating 10 at a time. Increase the temperature as well
    response_message = call_llm(llm_model, messages, tools,temperature=0.9)
    #print(response_message)
    tool_calls = response_message.tool_calls
    if tool_calls:
        #print(tool_calls)
        for tool_call in tool_calls:
            if tool_call.function.name in available_functions:
                function_to_call = available_functions[tool_call.function.name]
                function_args = json.loads(tool_call.function.arguments)
                AI_params = [param for param in function_args]
                Function_Params = list(list(available_functions[tool_call.function.name].__code__.co_varnames)[:available_functions[tool_call.function.name].__code__.co_argcount])
                Args_Match = set(AI_params) == set(Function_Params)
                #print(Args_Match)
                if Args_Match:
                    function_response = function_to_call(
                        Neighborhood=function_args.get("Neighborhood"),
                        Price=function_args.get("Price"),
                        Bedrooms=function_args.get("Bedrooms"),
                        Bathrooms=function_args.get("Bathrooms"),
                        House_Size=function_args.get("House_Size"),
                        Description=function_args.get("Description"),
                        Neighborhood_Description=function_args.get("Neighborhood_Description"),
            )
                    print(function_response)
                else:
                    print("Missing parameters")
    print(Real_Estate_Listings.count())


Brighton $1,200,000 4 3 2500sqft Luxurious 4-bedroom mansion with panoramic ocean views and a private swimming pool. This modern property features high-end finishes and a spacious interior, perfect for entertaining guests. Brighton is a prestigious neighborhood known for its upscale dining options, boutique shops, and waterfront promenade.
2393d7c0-382d-4df6-950a-396dc6c7debd has been added to the Real Estate Listings
South Yarra $1,800,000 3 2 2000sqft Charming 3-bedroom Victorian home with a beautifully landscaped garden. This heritage property exudes character and elegance, with original stained glass windows and period features throughout. South Yarra is a vibrant neighborhood with trendy cafes, art galleries, and easy access to public transport.
51b9fafe-8961-4f6e-9e26-39d1d7052e62 has been added to the Real Estate Listings
St Kilda $900,000 2 1 1500sqft Cozy 2-bedroom beach cottage just steps away from St Kilda Beach. This quaint home offers a relaxed coastal living experience wi

In [7]:
Real_Estate_Listings.peek(2)

{'ids': ['00e34b5f-177e-48db-aa0a-0681bfd95659',
  '0458e098-697b-4a47-97d5-70c3e01bba0f'],
 'embeddings': [[-0.006721713580191135,
   0.0202235896140337,
   0.0019645588472485542,
   -0.021730292588472366,
   -0.00686459057033062,
   0.01563853770494461,
   -0.017482949420809746,
   -0.004166163504123688,
   -0.007254255004227161,
   -0.007676391862332821,
   -0.010852158069610596,
   0.03722595423460007,
   0.009007745422422886,
   -0.025795793160796165,
   0.018742864951491356,
   0.0127030648291111,
   0.030939364805817604,
   -0.023977359756827354,
   0.002592892851680517,
   -0.016145100817084312,
   -0.036628469824790955,
   -0.006936029065400362,
   0.006332049146294594,
   -0.0007354918634518981,
   -0.005614416673779488,
   -0.02782204933464527,
   -0.009345455095171928,
   -0.018184345215559006,
   -0.0009522427571937442,
   0.01824929006397724,
   0.01574244722723961,
   -0.008955790661275387,
   -0.013352504931390285,
   -0.0012672216398641467,
   -0.007559492252767086,
  

# Conclusion and Observations
    1. Don't try to get the one data at a time, when variety is needed. as LLM doesn't have the history. It can likely repeat.
    2. Temperature is important parameter. Should be carefully used.
    3. When calling functions, First give the description & then give example. If you don't give description, Most likely function parameter will default to example. Also in the prompt, be explicit not to use the example.
    4. Example is important as well. When I used Melbourne as Neighborhood, It responded with answers such as Sydney, which is not in VIC. when I provided Williams Landing, It is able to provide right Neighborhoods.
     